## 7.1 문자열을 날짜로 변환하기
판다스의 to_datetime 함수를 사용하고 format 매개변수에서 날짜와 시간 포맷을 지정합니다.

In [2]:
import numpy as np
import pandas as pd

date_strings = np.array(['03-04-2005 11:35 PM',
                         '23-05-2010 12:01 AM',
                         '04-09-2009 09:09 PM'])

# Timestamp 객체로 바꿉니다.
[pd.to_datetime(date, format='%d-%m-%Y %I:%M %p') for date in date_strings]

[Timestamp('2005-04-03 23:35:00'),
 Timestamp('2010-05-23 00:01:00'),
 Timestamp('2009-09-04 21:09:00')]

오류 처리를 위해 errors 매개변수를 추가할 수도 있습니다.

In [3]:
# datetime으로 바꿉니다.
[pd.to_datetime(date, format="%d-%m-%Y %I:%M %p", errors ="ignore") for date in date_strings]

[Timestamp('2005-04-03 23:35:00'),
 Timestamp('2010-05-23 00:01:00'),
 Timestamp('2009-09-04 21:09:00')]

errors="coerce"는 문제가 발생해도 에러를 일으키지 않지만 대신 에러가 난 값을 NaT(즉 누락된 값)으로 설정합니다.

날짜와 시간이 문자열일 때 이를 파이썬이 인식할 수 있는 데이터 타입으로 바꾸어야 합니다. 문자열을 datetime으로 바꿀 수 있는 파이썬 도구가 많이 있지만 다른 레시피에서 판다스를 사용하는 것에 맞추어 to_datetime으로 변환을 수행했습니다. 날짜와 시간 문자열을 다룰 때 어려운 점은 문자열의 포맷이 데이터 소스마다 크게 다르다는 것입니다. 예를 들어 어떤 날짜 벡터는 March 23rd, 2015를 03-23-15로 나타낼 수 있고 다른 경우는 3|23|2015일 수 있습니다. format 매개변수를 사용하여 적합한 문자열의 포맷을 지정해야 합니다. 다음은 많이 사용하는 날짜와 시간 포맷 코드입니다.

|코드|설명                         | 예 | 
|----|:----------------------------|:--:|
|%Y  |전체 연도                    |2001|
|%m  |0으로 시작하는 월            |04  |
|%d  |0으로 시작하는 일            |09  |
|%I  |0으로 시작하는 시간(12시간제)|02  |
|%p  |AM 또는 PM                   |AM  |
|%M  |0으로 시작하는 분            |05  |
|%S  |0으로 시작하는 초            |09  |

NaT(Not a Time)는 판다스의 datetime 데이터 타입에서 누락된 값을 의미합니다. to_datetime 함수의 errors 매개변수의 기본값은 'raise'로 날짜 포맷에 문제가 있을 때 예외를 발생시킵니다. 또 다른 옵션으로 'ignore'는 예외를 발생시키거나 NaT를 반환하는 대신 원본 문자열을 그대로 반환합니다.

해결에서처럼 리스트 컴프리헨션을 사용하지 않고 to_datetime 함수에 date_strings 리스트를 그대로 전달할 수 있습니다. 또한 format 매개변수를 지정하지 않아도 가능한 날짜 포맷을 추측하여 변환해 줍니다.

In [4]:
pd.to_datetime(date_strings)

DatetimeIndex(['2005-03-04 23:35:00', '2010-05-23 00:01:00',
               '2009-04-09 21:09:00'],
              dtype='datetime64[ns]', freq=None)

## 7.2 시간대 다루기
특별히 지정하지 않으면 판다스 객체에는 시간대가 없습니다. 객체를 만들 때 tz 매개변수를 사용하여 시간대를 추가할 수 있습니다.

In [5]:
import pandas as pd

# datetime을 만듭니다.
pd.Timestamp('2017-05-01 06:00:00', tz='Europe/London')

Timestamp('2017-05-01 06:00:00+0100', tz='Europe/London')

tz_localize 메서드를 사용해 이전에 만든 datetime에 시간대를 추가할 수 있습니다.

In [7]:
# datetime을 만듭니다.
date = pd.Timestamp('2017-05-01 06:00:00')

# 시간대를 지정합니다.
date_in_london = date.tz_localize('Europe/London')

# datetime을 확인합니다.
date_in_london

Timestamp('2017-05-01 06:00:00+0100', tz='Europe/London')

다른 시간대로 변환할 수도 있습니다.

In [8]:
# 시간대를 바꿉니다. 
date_in_london.tz_convert('Africa/Abidjan')

Timestamp('2017-05-01 05:00:00+0000', tz='Africa/Abidjan')

마지막으로 판다스의 Series 객체는 모든 원소에 tz_localize와 tz_convert를 적용합니다.

In [9]:
# 세 개의 날짜를 만듭니다.
dates = pd.Series(pd.date_range('2/2/2002', periods=3, freq='M'))

# 시간대를 지정합니다.
dates.dt.tz_localize('Africa/Abidjan')

0   2002-02-28 00:00:00+00:00
1   2002-03-31 00:00:00+00:00
2   2002-04-30 00:00:00+00:00
dtype: datetime64[ns, Africa/Abidjan]

판다스는 두 종류의 시간대 문자열을 지원합니다. 하지만 pytz 라이브러리의 문자열을 사용하는 것이 좋습니다. all_timezones을 임포트하여 전체 시간대 문자열을 확인할 수 있습니다.

In [10]:
from pytz import all_timezones

all_timezones[0:2]

['Africa/Abidjan', 'Africa/Accra']

'dateutil/'로 시작하여 dateutil 문자열을 사용할 수 있습니다.

In [12]:
dates.dt.tz_localize('dateutil/Asia/Seoul')

0   2002-02-28 00:00:00+09:00
1   2002-03-31 00:00:00+09:00
2   2002-04-30 00:00:00+09:00
dtype: datetime64[ns, tzfile('ROK')]

pytz의 객체를 직접 전달할 수도 있습니다.

In [14]:
import pytz

tz = pytz.timezone('Asia/Seoul')
dates.dt.tz_localize(tz)

0   2002-02-28 00:00:00+09:00
1   2002-03-31 00:00:00+09:00
2   2002-04-30 00:00:00+09:00
dtype: datetime64[ns, Asia/Seoul]

## 7.3 날짜와 시간 선택하기
시작과 마지막 날짜를 사용해 불리언 조건을 만듭니다.

In [15]:
import pandas as pd

dataframe = pd.DataFrame()

dataframe['date'] = pd.date_range('1/1/2001', periods=100000, freq='H')

dataframe[(dataframe['date'] > '2002-1-1 01:00:00') & 
          (dataframe['date'] <= '2002-1-1 04:00:00')]

,date
8762,2002-01-01 02:00:00
8763,2002-01-01 03:00:00
8764,2002-01-01 04:00:00


또는 날짜 열을 데이터프레임의 인덱스로 지정하고 loc를 사용해 슬라이싱할 수 있습니다.

In [16]:
dataframe = dataframe.set_index(dataframe['date'])

dataframe.loc['2002-1-1 01:00:00':'2002-1-1 04:00:00']

,date
date,
2002-01-01 01:00:00,2002-01-01 01:00:00
2002-01-01 02:00:00,2002-01-01 02:00:00
2002-01-01 03:00:00,2002-01-01 03:00:00
2002-01-01 04:00:00,2002-01-01 04:00:00


불리언 조건을 사용할지 인덱스 슬라이싱을 사용할지는 상황에 따라 다릅니다. 복잡한 시계열 데이터를 다루어야 한다면 날짜 열을 데이터프레임의 인덱스로 지정할 만 합니다. 간단한 데이터 랭글링이라면 불리언 조건이 더 쉽습니다.